In [4]:
import pandas as pd

retail_data = pd.read_csv('/content/OnlineRetail.csv', encoding= 'unicode_escape')

retail_data.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
434094,574029,22460,EMBOSSED GLASS TEALIGHT HOLDER,24,11/2/2011 12:34,1.25,12955.0,United Kingdom
387034,570258,22379,RECYCLING BAG RETROSPOT,3,10/10/2011 9:56,4.96,NaN,United Kingdom
125683,547057,22829,SWEETHEART WIRE WALL TIDY,2,3/20/2011 12:28,9.95,14217.0,United Kingdom
26983,538524,22207,FRYING PAN UNION FLAG,1,12/13/2010 9:35,8.47,NaN,United Kingdom
286951,562046,23249,VINTAGE RED ENAMEL TRIM PLATE,12,8/2/2011 10:34,1.65,12378.0,Switzerland


# **Análise Exploratória**

Antes de iniciar a feature engineering e a construção do modelo, cientistas de dados fazem uma análise exploratória.

Nela a ideia é checar se o dataset possui dados suficientes para resolver o problema, identificando gaps e se alguma correlação no dataset, entre outros.

Para este primeiro exercício, vamos calcular a venda mensal e verificar a sazionalidade.

In [5]:
## Convert 'InvoiceDate' to of type datatime

retail_data['InvoiceDate'] = pd.to_datetime(
    retail_data['InvoiceDate'], errors = 'coerce'
)

##Extract year and month information from 'InvoiceDate'

retail_data['yyyymm'] = retail_data['InvoiceDate'].dt.strftime('%Y%m')

## Calculate revenue generated per order
retail_data['revenue'] = retail_data['UnitPrice'] * retail_data['Quantity']

## Calculate monthly revenue by aggregating the revenue on year month column

revenue_df = retail_data.groupby(['yyyymm'])['revenue'].sum().reset_index()
revenue_df.head()

,yyyymm,revenue
0,201012,748957.020
1,201101,560000.260
2,201102,498062.650
3,201103,683267.080
4,201104,493207.121


In [7]:
import plotly.express as px

revenue_df.sort_values(by=['yyyymm'], inplace=True)

fig = px.bar(revenue_df, x='yyyymm', y='revenue', title='Monthly Revenue')
fig.update_xaxes(type='category')
fig.show()

In [8]:
active_customer_df = retail_data.groupby(['yyyymm'])['CustomerID'].nunique().reset_index()

active_customer_df.columns = ['yyyymm', 'No of Active customers']

active_customer_df.head()

,yyyymm,No of Active customers
0,201012,948
1,201101,783
2,201102,798
3,201103,1020
4,201104,899


In [9]:
fig = px.bar(active_customer_df, x='yyyymm', y='No of Active customers', title='Monthly Active customers')
fig.update_xaxes(type='category')
fig.show()

# **Modelo**

# Customer Lifetime Value (CLTV)

Um jeito de prever o valor de um cliete é prever o grupo do cliente.
O cliente será dividido em 3 grupos:


1.   Low LTV
2.   Mid LTV
3.   High LTV

Vamos utilizar 3 meses dos daos para cacular Recency(R), Frequency(F) e Monetary(M) para gerar as features. Assim que consguirmos as features iremos utilizar 6 meses dos dados para calcular a venda de cada consumidor e gerar os clustes de labels LTV. Assim as labels geradas serão usadas para treinar o modelo XGBoost que será usado para prever o grupo de novos consumidores



# Feature Engineering

1. Recency(R) - Quantos dias passou desde a última compra do cliente
2. Frequency (F) - Quantas vezes um consumidor fez uma compra
3. Monetary (M) - Quando de receita um consumidor gerou

Como os padrões de compra variam de acordo com a região, utilizaremos apenas os dados do Reino Unido

In [11]:
import pandas as pd
from datetime import datetime, timedelta, date
from sklearn.cluster import KMeans

retail_data = pd.read_csv('/content/OnlineRetail.csv', encoding='unicode_escape')
retail_data['InvoiceDate'] = pd.to_datetime(retail_data['InvoiceDate'], errors = 'coerce')

uk_data = retail_data.query("Country=='United Kingdom'").reset_index(drop=True)

In [14]:
t1 = pd.Timestamp("2011-06-01 00:00:00.054000")
t2 = pd.Timestamp("2011-03-01 00:00:00.054000")
t3 = pd.Timestamp("2011-12-01 00:00:00.054000")

uk_data_3m = uk_data[(uk_data.InvoiceDate < t1) & (uk_data.InvoiceDate >= t2)].reset_index(drop=True)
uk_data_6m = uk_data[(uk_data.InvoiceDate >= t1) & (uk_data.InvoiceDate < t3)].reset_index(drop=True)

In [15]:
uk_data_3m['revenue'] = uk_data_3m['UnitPrice']*uk_data_3m['Quantity']

max_date = uk_data_3m['InvoiceDate'].max() + timedelta(days=1)
rfm_data = uk_data_3m.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (max_date - x.max()).days,
    'InvoiceNo': 'count',
    'revenue': 'sum'
})

rfm_data.rename(columns={'InvoiceDate': 'Recency',
                         'InvoiceNo': 'Frequency',
                         'revenue': 'MonetaryValue'}, inplace=True)

In [17]:
r_grp = pd.qcut(rfm_data['Recency'], q=4, labels=range(3, -1, -1))
f_grp = pd.qcut(rfm_data['Frequency'], q=4, labels=range(0, 4))
m_grp = pd.qcut(rfm_data['MonetaryValue'], q=4, labels=range(0, 4))

rfm_data = rfm_data.assign(R=r_grp.values).assign(F=f_grp.values).assign(M=m_grp.values)
rfm_data['R'] = rfm_data['R'].astype(int)
rfm_data['F'] = rfm_data['F'].astype(int)
rfm_data['M'] = rfm_data['M'].astype(int)

rfm_data['RFMScore'] = rfm_data['R'] + rfm_data['F'] + rfm_data['M']
rfm_data.groupby('RFMScore')['Recency', 'Frequency', 'MonetaryValue'].mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,Recency,Frequency,MonetaryValue
RFMScore,,,
0,76.633588,5.206107,94.023359
1,58.500000,9.074074,158.990864
2,54.380208,13.968750,239.346354
3,38.091633,16.605578,271.437331
4,36.647619,24.180952,363.867000
5,34.000000,33.558252,535.423204
6,28.516432,44.920188,742.487324
7,18.431138,51.317365,974.310120
8,14.316547,80.064748,2192.578568


In [18]:
rfm_data['Segment'] = 'Low-Value'
rfm_data.loc[rfm_data['RFMScore']>4, 'Segment'] = 'Mid-Value'
rfm_data.loc[rfm_data['RFMScore']>6, 'Segment'] = 'High-Value'
rfm_data = rfm_data.reset_index()

# Customer LTV

In [20]:
uk_data_6m['revenue'] = uk_data_6m['UnitPrice'] * uk_data_6m['Quantity']
revenue_6m = uk_data_6m.groupby(['CustomerID']).agg({
    'revenue': 'sum'
})
revenue_6m.rename(columns={'revenue': 'Revenue_6m'},
                  inplace=True)
revenue_6m = revenue_6m.reset_index()
revenue_6m = revenue_6m.dropna()

merged_data = pd.merge(rfm_data, revenue_6m, how="left")
merged_data.fillna(0)

,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFMScore,Segment,Revenue_6m
0,12747.0,7,35,1082.09,3,2,3,8,High-Value,1666.11
1,12748.0,1,582,4336.73,3,3,3,9,High-Value,18679.01
2,12749.0,8,54,782.10,3,3,3,9,High-Value,2323.04
3,12821.0,23,6,92.72,2,0,0,2,Low-Value,0.00
4,12823.0,63,1,459.00,0,0,2,2,Low-Value,765.00
...,...,...,...,...,...,...,...,...,...,...
1835,18272.0,21,59,966.74,2,3,3,8,High-Value,1730.16
1836,18273.0,66,1,51.00,0,0,0,0,Low-Value,102.00
1837,18280.0,86,10,180.60,0,0,0,0,Low-Value,0.00
1838,18283.0,9,100,217.15,3,3,1,7,High-Value,1351.83


In [22]:
merged_data = merged_data[merged_data['Revenue_6m']<merged_data['Revenue_6m'].quantile(0.99)]
kmeans = KMeans(n_clusters=3)
kmeans.fit(merged_data[['Revenue_6m']])
merged_data['LTVCluster'] = kmeans.predict(merged_data[['Revenue_6m']])
merged_data.groupby('LTVCluster')['Revenue_6m'].describe()

,count,mean,std,min,25%,50%,75%,max
LTVCluster,,,,,,,,
0,993.0,629.803375,428.475057,-609.4,294.8200,550.70,943.3600,1590.15
1,312.0,2553.449167,820.919021,1600.0,1864.2875,2272.31,3078.5800,4558.22
2,58.0,6739.215172,1747.158280,4707.4,5497.9800,6228.34,7358.3175,11730.37


# Feature set e modelo


In [23]:
feature_data = pd.get_dummies(merged_data)
feature_data.head(5)

,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFMScore,Revenue_6m,LTVCluster,Segment_High-Value,Segment_Low-Value,Segment_Mid-Value
0,12747.0,7,35,1082.09,3,2,3,8,1666.11,1,1,0,0
2,12749.0,8,54,782.10,3,3,3,9,2323.04,1,1,0,0
4,12823.0,63,1,459.00,0,0,2,2,765.00,0,0,1,0
7,12836.0,28,62,814.71,1,3,3,7,951.46,0,1,0,0
8,12839.0,7,24,303.58,3,2,1,6,3443.14,1,0,0,1


In [25]:
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split

X = feature_data.drop(['CustomerID', 'LTVCluster', 'Revenue_6m'], axis=1)
y = feature_data['LTVCluster']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
xgb_classifier = xgb.XGBClassifier(max_depth=5, objective='multi:softprob')
xgb_model = xgb_classifier.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89       104
           1       0.48      0.60      0.54        25
           2       0.50      0.12      0.20         8

    accuracy                           0.80       137
   macro avg       0.63      0.54      0.54       137
weighted avg       0.80      0.80      0.79       137

